<a href="https://colab.research.google.com/github/sseunghyuns/deeplearning/blob/master/cats_vs_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision with CNNs

## 라이브러리 불러오기

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
dataset_name = 'cats_vs_dogs'

train_dataset, info = tfds.load(name=dataset_name, split = 'train[:80%]', with_info=True)
valid_dataset = tfds.load(name=dataset_name, split = 'train[-20%:]')

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete7TMSTV/cats_vs_dogs-train.tfrecord



Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


## 데이터 전처리

In [3]:
def preprocess(features):
    # YOUR CODE HERE
    img, lbl = tf.cast(features['image'], tf.float32) / 255.0, features['label']
    img = tf.image.resize(img, size = (224, 224))
    return img, lbl

* 데이터를 image, label로 나누는 함수를 정의하였습니다.
* image 데이터를 0~1사이 값으로 정규화하였습니다.
* image 데이터 크기를 (224,224)으로 통일하였습니다.

In [4]:
batch_size = 32
total_size = info.splits['train'].num_examples
validation_steps= (total_size *0.2) // batch_size +1
steps_per_epoch= (total_size * 0.8) // batch_size +1


train_data = train_dataset.map(preprocess).batch(batch_size).repeat()
valid_data = valid_dataset.map(preprocess).batch(batch_size).repeat()

* 데이터 전처리 함수를 적용하고 배치 사이즈만큼 학습을 할 수 있도록 하였습니다.

## CNN 모델 생성

In [5]:
model = Sequential([
                    Conv2D(128, (3,3), input_shape = (224,224,3), activation = 'relu'),# 222 x 222 x 3 x 128
                    MaxPooling2D(), # 111 x 111 x 3 x 128

                    Conv2D(64, (3,3), activation = 'relu'), # 109 x 109 x 3 x 64
                    MaxPooling2D(), # 54 x 54 x 3 x 64

                    Conv2D(32, (3,3), activation = 'relu'), # 52 x 52 x 3 x 32
                    MaxPooling2D(), # 26 x 26 x 3 x 32

                    Conv2D(16, (3,3), activation = 'relu'), # 24 x 24 x 3 x 16
                    MaxPooling2D(), # 12 x 12 x 3 x 16

                    Flatten(),
                    Dropout(0.5),

                    Dense(64, activation = 'relu'),
                    Dense(32, activation = 'relu'),
                    Dense(2, activation='softmax')
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 128)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 16)        4

In [6]:
# early_stopper
#early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10, verbose = 1)

#optimizer = tf.keras.optimizers.RMSprop(0.001)

In [7]:
# 모델 compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='acc')


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', # 검증 손실을 기준으로 callback이 호출
                                                  factor=0.5, # callback 호출시 학습률을 1/2로 줄임.
                                                  patience=3, # epoch 10 동안 개선되지 않으면 callback이 호출
                                                  min_lr=0.000001)

# 모델 체크포인트
check_path = 'ck.ckpt'
check_point = ModelCheckpoint(filepath = check_path,
                              monitor='val_loss', 
                              save_best_only=True, 
                              save_weights_only=True, 
                              verbose = 1)

## 모델 학습

In [8]:

# 모델 학습
model.fit(train_data,
          validation_data = (valid_data),
          steps_per_epoch = steps_per_epoch,
          validation_steps = validation_steps,
          epochs = 40,
          callbacks=[check_point,reduce_lr]
)

Epoch 1/40
  2/582 [..............................] - ETA: 16s - loss: 0.6959 - acc: 0.5781WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0227s vs `on_train_batch_end` time: 0.0395s). Check your callbacks.


582/582 [==============================] - ETA: 0s - loss: 0.6683 - acc: 0.5884
Epoch 00001: val_loss improved from inf to 0.62140, saving model to ck.ckpt
582/582 [==============================] - 41s 71ms/step - loss: 0.6683 - acc: 0.5884 - val_loss: 0.6214 - val_acc: 0.6696
Epoch 2/40
582/582 [==============================] - ETA: 0s - loss: 0.5952 - acc: 0.6814
Epoch 00002: val_loss improved from 0.62140 to 0.57066, saving model to ck.ckpt
582/582 [==============================] - 43s 74ms/step - loss: 0.5952 - acc: 0.6814 - val_loss: 0.5707 - val_acc: 0.7266
Epoch 3/40
581/582 [============================>.] - ETA: 0s - loss: 0.5289 - acc: 0.7387
Epoch 00003: val_loss improved from 0.57066 to 0.48910, saving model to ck.ckpt
582/582 [==============================] - 43s 73ms/step - loss: 0.5288 - acc: 0.7387 - val_loss: 0.4891 - val_acc: 0.7633
Epoch 4/40
582/582 [==============================] - ETA: 0s - loss: 0.4668 - acc: 0.7785
Epoch 00004: val_loss improved from 0.4891

In [9]:
# 모델 load
model.load_weights(check_path)

* 성능이 가장 좋았던 weigth를 로드하고 모델을 저장합니다. 